In [1]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pathlib import Path
from collections import Counter, defaultdict
import pandas as pd
import random
from tqdm.notebook import tqdm
import nltk
from nltk.util import ngrams
import nltk.data
import re
import contractions
from bs4 import BeautifulSoup
import unidecode
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
import math
import numpy as np
import copy
import seaborn as sns
import matplotlib.pyplot as plt

nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
])

lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()
encoder = LabelEncoder()
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
tqdm.pandas()

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up

In [3]:
computing = pd.read_csv("Datasets\\KhanAcademy\\Computing.csv")
computing = computing.dropna()

economics = pd.read_csv("Datasets\\KhanAcademy\\Economics.csv")
economics = economics.dropna()

humanities = pd.read_csv("Datasets\\KhanAcademy\\Humanities.csv")
humanities = humanities.dropna()

math = pd.read_csv("Datasets\\KhanAcademy\\Math.csv")
math = math.dropna()

science = pd.read_csv("Datasets\\KhanAcademy\\Science.csv")
science = science.dropna()

khan_dfs = [computing, economics, humanities, math, science]
khan = pd.concat(khan_dfs, axis=0)

In [4]:
def clean_base(x: str):
        # remove any html tags
        x = BeautifulSoup(x, "html.parser").get_text(separator=" ")
        # # set all to lower
        # x = x.lower()
        # clean up the contractions
        x = contractions.fix(x)
        # remove accended characters
        x = unidecode.unidecode(x)
        # # remove stopwords: https://stackoverflow.com/questions/19560498/faster-way-to-remove-stop-words-in-python
        # x = ' '.join([word for word in x.split() if word not in cachedStopWords]) # slower to use word tokenize
        # # fix punctuation spacing
        # x = re.sub(r'(?<=[\.\,\?])(?=[^\s])', r' ', x)
        # # strip punctuation
        # x = re.sub(r'[\.\,\?\\\/\<\>\;\:\[\]\{\}]', r'', x)
        # strip quotes
        x = x.replace('\'', '').replace('\"', '')
        # remove some actions
        remove_list = ['(Laughter)', '(laughter)', '(Music)', '(music)', '(Music ends)', '(Audience cheers)', '(Applause)', '(Applause ends)', '(Applause continues)', '(Bells)', '(Trumpet)', '(Clears throat)']
        x = ' '.join([word for word in x.split() if word not in remove_list])
        # remove extraneous items
        x = x.replace(' -- ', '').replace(' .. ', ' ').replace(' ... ', ' ')
        # remove extra whitespace
        x = ' '.join(x.strip().split())
        # # may want to add lematization
        # x = ' '.join([lemmatizer.lemmatize(word) for word in x.split()])
        # remove some of the extra bracket tags
        x = re.sub(r"\s{2,}", " ", re.sub(r"[\(\[\{][^\)\]\}]*[\)\]\}]", "", x))
        return x

def remove_first_sentence(doc):
    """
    This removes the first sentence of a document. We use this to remove all narrator / speaker tags, and
    to remove unnecessary introductory sentences that most transcripts have
    """
    return ' '.join(nltk.sent_tokenize(doc)[1:])

transcripts = khan['transcript']
transcripts = transcripts.progress_apply(remove_first_sentence)
transcripts = transcripts.progress_apply(clean_base)
khan['clean_transcript'] = transcripts
khan.head()


  0%|          | 0/8261 [00:00<?, ?it/s]

  0%|          | 0/8261 [00:00<?, ?it/s]

,course,unit,lesson,video_title,about,transcript,clean_transcript
0,Computer programming,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...","If you have never learned to program before, y..."
1,Computer programming,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,Voiceover: Ok so you've\r\nmade a few programs...,"Is it width and height, or is it height and wi..."
5,Computer programming,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"- [Voiceover] On Khan Academy, we pop up the o...",But we only tell you about the big things. The...
6,Computer programming,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,- [Instructor] The world is full of data. Ever...,Every app that you use is full of data. On Kha...
7,Computer programming,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"At this point, you've probably heard me\r\npro...",Some of you might even be mad that I am pronou...


In [5]:
sat = pd.read_csv("Datasets\\SAT_Question_Dataset.csv")
sat.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        152 non-null    int64 
 1   ans       152 non-null    object
 2   question  152 non-null    object
 3   a)        152 non-null    object
 4   b)        152 non-null    object
 5   c)        152 non-null    object
 6   d)        152 non-null    object
 7   e)        152 non-null    object
 8   year      152 non-null    int64 
 9   sec       152 non-null    int64 
 10  num       152 non-null    int64 
 11  diff      152 non-null    object
 12  blanks    152 non-null    int64 
dtypes: int64(5), object(8)
memory usage: 15.6+ KB


In [6]:
for column_name in ['question', 'a)', 'b)', 'c)', 'd)', 'e)']:
    sat[column_name] = sat[column_name].progress_apply(clean_base)
sat.head()

  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/152 [00:00<?, ?it/s]

,id,ans,question,a),b),c),d),e),year,sec,num,diff,blanks
0,1,e,Much of our knowledge of dinosaurs comes from ...,Much of our knowledge of dinosaurs comes from ...,Much of our knowledge of dinosaurs comes from ...,Much of our knowledge of dinosaurs comes from ...,Much of our knowledge of dinosaurs comes from ...,Much of our knowledge of dinosaurs comes from ...,2001,1,1,1,2
1,2,c,Responding to criticism that the script was ra...,Responding to criticism that the script was ra...,Responding to criticism that the script was ra...,Responding to criticism that the script was ra...,Responding to criticism that the script was ra...,Responding to criticism that the script was ra...,2001,1,2,1,2
2,3,e,Vernal pools are among the most _____ of ponds...,Vernal pools are among the most transitory of ...,Vernal pools are among the most anachronistic ...,Vernal pools are among the most immutable of p...,Vernal pools are among the most itinerant of p...,Vernal pools are among the most ephemeral of p...,2001,1,3,2,2
3,4,e,"During the 1990s, Shanghai benefited from an a...","During the 1990s, Shanghai benefited from an a...","During the 1990s, Shanghai benefited from an a...","During the 1990s, Shanghai benefited from an a...","During the 1990s, Shanghai benefited from an a...","During the 1990s, Shanghai benefited from an a...",2001,1,4,3,1
4,5,b,Many subatomic nuclear particles are _____ and...,Many subatomic nuclear particles are unstable ...,Many subatomic nuclear particles are elusive a...,Many subatomic nuclear particles are minute an...,Many subatomic nuclear particles are charged a...,Many subatomic nuclear particles are tenuous a...,2001,1,5,3,2


In [7]:
# create a list of all of the sentences with each sentence tokenized.
def tokenize_transcript(transcript):
    token_document = [word_tokenize(t) for t in sent_detector.tokenize(transcript)]
    sents = []
    start_of_sentence = "<sent>"
    end_of_sentence = "<\\sent>"
    for sent in token_document:
        sent.insert(0, start_of_sentence)
        sent.append(end_of_sentence)
        sents.append(sent)
    return sents

sents = []
for transcript in tqdm(khan['clean_transcript']):
    sents += tokenize_transcript(transcript)

print(len(sents))

  0%|          | 0/8261 [00:00<?, ?it/s]

603547


In [8]:
n = 3
print("Computing all N-grams where N is %d or less" % n)
grams = []
for sent in tqdm(sents):
    for i in range(0, 3):
        grams += list(ngrams(sent, i + 1))
    
print("Computing frequency distribution")
freq_dist = nltk.FreqDist(grams)

Computing all N-grams where N is 3 or less


  0%|          | 0/603547 [00:00<?, ?it/s]

Computing frequency distribution


In [21]:
def tokenize_question(question):
    return tokenize_transcript(question)[0]

def get_mask_indices(row):
    mask_indices = []
    previous_idx = 0
    for blank_id in range(row['blanks']):
        idx = tokenize_question(row['question']).index(mask, previous_idx)
        mask_indices.append(idx)
        previous_idx = idx + 1
    return mask_indices

def extract_possible_solutions(row, mask_indices):
    possible_solutions = []
    for i in ['a)', 'b)', 'c)', 'd)', 'e)']:
        # print(test[i])
        tokens = tokenize_question(row[i])
        possible_solution = []
        for mask_idx in mask_indices:
            possible_solution.append(tokens[mask_idx])
        possible_solutions.append(tuple(possible_solution))
    return possible_solutions

def get_ngrams_with_mask(mask_indices, tokenized_question, n):
    """This method is used to compute the sliding window for the original n-gram size as well
    as each of the subsequently smaller sizes of n. This method requires the variable 'n' to
    be defined globally for the maximum n-gram size to look for.

    Args:
        row : a row from a pandas dataframe of the SAT dataset to create the masks from.

    Returns:
        list: A list containing tuples of each n-gram is returned. If more than one 
    """
    ranges = []
    count = 0
    for mask_idx in mask_indices:
        remask = mask + str(count)
        tokenized_question[mask_idx] = remask
        count += 1
        # print("\nmask_idx = %d" % mask_idx)
        for i in range(n, 0, -1):  # for each successively smaller n-gram size
            # print("n = %d" % i)
            upper_bound = mask_idx + 1
            lower_bound = mask_idx - i + 1
            if lower_bound >= 0 and upper_bound < len(tokenized_question):
                ranges.append((lower_bound, upper_bound))
                # print("range(%d, %d)" % (lower_bound, upper_bound))
            for j in range(1, i):  # already processed first n-gram for size i. Now process the rest where j is the offset.
                upper_bound += 1
                lower_bound += 1
                if lower_bound >= 0 and upper_bound < len(tokenized_question):
                    ranges.append((lower_bound, upper_bound))
                    # print("range(%d, %d)" % (lower_bound, upper_bound))
    n_grams = []
    for indices in ranges:
        n_gram = []
        for i in range(indices[0], indices[1]):
            n_gram.append(tokenized_question[i])
        n_grams.append(n_gram)
        
    return n_grams
    
    
print("compute the windows")
test = sat.iloc[1]
mask = "_____"
mask_indices = get_mask_indices(test)
tokenized_question = tokenize_question(test['question'])
possible_solutions = extract_possible_solutions(test, mask_indices)
windows = get_ngrams_with_mask(mask_indices, tokenized_question, n=3)
print("mask indicies = %s" % str(mask_indices))
print(possible_solutions)
print(windows)
print(tokenized_question)

compute the windows
mask indicies = [10, 22]
[('engaging', 'simplicity'), ('subjective', 'ambiguity'), ('muddled', 'clarity'), ('terse', 'emptiness'), ('difficult', 'abstraction')]
[['rambling', 'and', '_____0'], ['and', '_____0', ','], ['_____0', ',', 'the'], ['and', '_____0'], ['_____0', ','], ['_____0'], ['succinctness', 'and', '_____1'], ['and', '_____1', '.'], ['and', '_____1'], ['_____1', '.'], ['_____1']]
['<sent>', 'Responding', 'to', 'criticism', 'that', 'the', 'script', 'was', 'rambling', 'and', '_____0', ',', 'the', 'new', 'screenwriter', 'revised', 'the', 'dialogue', 'for', 'greater', 'succinctness', 'and', '_____1', '.', '<\\sent>']
